In [ ]:
#Import Packages
!pip install flwr
!pip install torch
!pip install matplotlib
!pip install torchvision
!pip install flwr-datasets[vision]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 523.6/523.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.68.1
    Uninstalling grpcio-1.68.1:
      Successfully uninstalled grpcio-1.68.1
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
  Attempting uninstall: typer
    Found existing installation: typer 0.15.1
    Uninstalling typer-0.15.1:
      Successfully uninstalled typer-0.15.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
!pip install -q flwr[simulation] flwr-datasets[vision] torch torchvision matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 16.0 MB/s eta 0:00:00


In [ ]:
# Import statements
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset, Dataset
from torchvision.datasets import MNIST
from collections import defaultdict
from typing import List, Tuple, Dict, Optional

import flwr as fl
from flwr.client import Client, NumPyClient
from flwr.server import ServerConfig
from flwr.server.strategy import FedAvg

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Device configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")


Training on cpu


In [ ]:
# Global Variables
NUM_CLIENTS = 10 # 10 Nodes
NUM_ROUNDS = 20
BATCH_SIZE = 32
INITIAL_ROUNDS = 5  # Minimum number of rounds to start initial selection
K = 3  # Number of clients to select per round
ALPHA_VALUES = [0.5, 0.2, 0.1]  # Different alpha values

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Load MNIST training data
trainset = MNIST(root='./data', train=True, download=True)

# Create a dictionary mapping class labels to indices
label_indices = defaultdict(list)
for idx, (img, label) in enumerate(trainset):
    label_indices[label].append(idx)

# Shuffle the indices within each label
for label in label_indices:
    np.random.shuffle(label_indices[label])

# Initialize client indices
client_indices = {i: [] for i in range(NUM_CLIENTS)}

# Assign images to clients 0 and 1 (labels 0,1,2)
for label in [0, 1, 2]:
    indices = label_indices[label]
    split = len(indices) // 2
    client_indices[0].extend(indices[:split])
    client_indices[1].extend(indices[split:])

# Assign images to clients 2 and 3 (labels 3,4,5)
for label in [3, 4, 5]:
    indices = label_indices[label]
    split = len(indices) // 2
    client_indices[2].extend(indices[:split])
    client_indices[3].extend(indices[split:])

# Assign images to clients 4 and 5 (labels 6,7,8,9)
for label in [6, 7, 8, 9]:
    indices = label_indices[label]
    split = len(indices) // 2
    client_indices[4].extend(indices[:split])
    client_indices[5].extend(indices[split:])

# Remaining images per label are assigned to clients 6-9
remaining_label_indices = {}
for label in range(10):
    indices = label_indices[label]
    split = len(indices) // 2
    remaining_label_indices[label] = indices[split:]

# Define client label distributions for clients 6-9
client_label_distributions = {
    6: [0.15]*5 + [0.05]*5,  # Client 6 favors labels 0-4
    7: [0.05]*5 + [0.15]*5,  # Client 7 favors labels 5-9
    8: [0.1]*10,             # Client 8 uniform distribution
    9: np.random.dirichlet(np.ones(10), size=1)[0].tolist(),  # Client 9 random distribution
}

# For clients 6-9, compute label counts and assign images
for client_id in [6, 7, 8, 9]:
    total_images_per_client = sum(len(remaining_label_indices[label]) for label in range(10)) // 4
    client_label_probs = client_label_distributions[client_id]
    client_label_counts = np.floor(np.array(client_label_probs) * total_images_per_client).astype(int)
    remaining = total_images_per_client - np.sum(client_label_counts)
    # Distribute the remaining counts
    for i in range(remaining):
        client_label_counts[i % 10] += 1

    # Now sample indices for this client
    for label in range(10):
        count = client_label_counts[label]
        indices = remaining_label_indices[label][:count]
        client_indices[client_id].extend(indices)
        remaining_label_indices[label] = remaining_label_indices[label][count:]

# Define a function to get the client-specific dataset
def get_client_trainset(client_id):
    indices = client_indices[client_id]
    return Subset(trainset, indices)

# Custom Dataset class to apply transformations
class TransformSubset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return {'img': x, 'label': y}

    def __len__(self):
        return len(self.subset)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 16.1MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 613kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.64MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 4.43MB/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# Function to load datasets for each client
def load_datasets(partition_id: int):
    # Get the client's dataset
    client_dataset = get_client_trainset(int(partition_id))

    # Split the client dataset into train/val sets: 80% train, 20% validation
    num_samples = len(client_dataset)
    indices = list(range(num_samples))
    np.random.shuffle(indices)

    split = int(np.floor(0.2 * num_samples))
    train_indices, val_indices = indices[split:], indices[:split]

    trainset = Subset(client_dataset, train_indices)
    valset = Subset(client_dataset, val_indices)

    # Define transformations for MNIST
    pytorch_transforms = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )

    # Apply transformations
    trainset = TransformSubset(trainset, transform=pytorch_transforms)
    valset = TransformSubset(valset, transform=pytorch_transforms)

    # Create DataLoaders
    trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
    valloader = DataLoader(valset, batch_size=BATCH_SIZE)

    # For the test set, use the standard MNIST test set
    testset = MNIST(root='./data', train=False, download=True, transform=pytorch_transforms)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)

    return trainloader, valloader, testloader

In [ ]:
# CNN Model for MNIST
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)   # 1 channel for grayscale
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)            # Flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
# Training function
def train(net, trainloader, epochs: int):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for _ in range(epochs):
        for batch in trainloader:
            images, labels = batch["img"].to(DEVICE), batch["label"].to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

# Testing function
def test(net, testloader):
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for batch in testloader:
            images, labels = batch["img"].to(DEVICE), batch["label"].to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [ ]:
# Functions to set and get model parameters
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = {k: torch.Tensor(v) for k, v in params_dict}
    net.load_state_dict(state_dict, strict=True)

def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

In [ ]:
# Flower client implementation
class FlowerClient(NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader.dataset), {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader.dataset), {"accuracy": float(accuracy)}

In [ ]:
# Client function
def client_fn(cid: str) -> Client:
    # Load model
    net = Net().to(DEVICE)
    # Load data
    trainloader, valloader, _ = load_datasets(cid)
    # Create and return the Flower client
    return FlowerClient(net, trainloader, valloader)

# **Method 6: Dynamic K and Alpha Selection(NON IID)**

This method dynamically adjusts client participation using K and alpha parameters to balance exploration and exploitation, optimizing accuracy and communication efficiency.

In [ ]:
import itertools
import csv
import flwr as fl
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy import Strategy
from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from typing import Dict, List, Optional, Tuple
import numpy as np
from flwr.common.logger import log
from logging import INFO, WARNING, ERROR
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Define necessary variables
NUM_CLIENTS = 10
INITIAL_ROUNDS = 5
NUM_ROUNDS = 20

class CS_final(Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
        evaluate_fn=None,
        on_fit_config_fn=None,
        on_evaluate_config_fn=None,
        accept_failures: bool = True,
        initial_rounds: int = 5,  # Minimum number of initial rounds
        k: int = 3,               # Number of clients to select per round
        alpha: float = 0.5,       # Smoothing factor for selection probability
        total_rounds: int = 20,   # Total number of rounds
    ):
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients
        self.evaluate_fn = evaluate_fn
        self.on_fit_config_fn = on_fit_config_fn
        self.on_evaluate_config_fn = on_evaluate_config_fn
        self.accept_failures = accept_failures

        self.initial_rounds = initial_rounds
        self.k = k
        self.alpha = alpha
        self.client_accuracies: Dict[str, float] = {}
        self.current_round = 0
        self.parameters: Optional[Parameters] = None  # Will be set after initialization
        self.global_accuracy: float = 0.0  # Store global model accuracy

        self.total_rounds = total_rounds
        self.total_communications = 0     # Initialize total communications

        # Track clients selected in initial rounds to ensure all participate at least once
        self.initial_selected_clients: set = set()

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        # Return None to let the server initialize parameters from a client
        return None

    def configure_fit(
        self,
        server_round: int,
        parameters: Parameters,
        client_manager: ClientManager,
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""
        self.current_round = server_round
        log(INFO, f"\n[ROUND {server_round}]")

        clients_dict = client_manager.all()
        available_cids = list(clients_dict.keys())

        # Ensure all clients have an accuracy value
        for cid in available_cids:
            if cid not in self.client_accuracies:
                self.client_accuracies[cid] = 0.0  # Default accuracy

        selected_cids = []

        if server_round <= self.initial_rounds:
            # Select top-k least accurate clients that haven't been selected yet
            # Sort clients by least accuracy
            sorted_clients = sorted(
                self.client_accuracies.items(), key=lambda x: x[1]
            )
            # Select clients not yet selected
            available_for_selection = [
                cid for cid, _ in sorted_clients if cid not in self.initial_selected_clients
            ]
            # Select up to k clients
            for cid, _ in sorted_clients:
                if cid not in self.initial_selected_clients:
                    selected_cids.append(cid)
                    self.initial_selected_clients.add(cid)
                    if len(selected_cids) == self.k:
                        break
            # If not enough unique clients, fill the rest with top-k least accurate
            if len(selected_cids) < self.k:
                additional = self.k - len(selected_cids)
                for cid, _ in sorted_clients:
                    if cid not in selected_cids:
                        selected_cids.append(cid)
                        if len(selected_cids) == self.k:
                            break
            log(
                INFO,
                f"Round {server_round}: Initial selection - Selected clients {selected_cids}",
            )
        else:
            # Probabilistic selection based on 1 - alpha * accuracy
            probabilities = {}
            for cid in available_cids:
                accuracy = self.client_accuracies[cid]
                prob = max(1 - self.alpha * accuracy, 0.0)
                probabilities[cid] = prob

            # Normalize probabilities
            total_prob = sum(probabilities.values())
            if total_prob == 0:
                # If total_prob is zero, default to uniform probabilities
                for cid in probabilities:
                    probabilities[cid] = 1.0 / len(probabilities)
            else:
                for cid in probabilities:
                    probabilities[cid] /= total_prob

            # Select k unique clients based on probabilities
            selected_cids = list(
                np.random.choice(
                    available_cids,
                    size=self.k,
                    replace=False,
                    p=[probabilities[cid] for cid in available_cids],
                )
            )
            log(
                INFO,
                f"Round {server_round}: Probabilistic selection - Selected clients {selected_cids}",
            )

        # Update total communications
        self.total_communications += len(selected_cids)

        clients = [
            clients_dict.get(cid) for cid in selected_cids if clients_dict.get(cid) is not None
        ]

        # Create fit instructions
        config = self.on_fit_config_fn(server_round) if self.on_fit_config_fn else {}
        fit_ins = FitIns(parameters, config)

        # Return client configurations as a list of (ClientProxy, FitIns)
        return [(client, fit_ins) for client in clients if client is not None]

    def configure_evaluate(
        self,
        server_round: int,
        parameters: Parameters,
        client_manager: ClientManager,
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        # Evaluate on all clients
        clients = list(client_manager.all().values())

        # Create evaluate instructions
        config = (
            self.on_evaluate_config_fn(server_round)
            if self.on_evaluate_config_fn
            else {}
        )
        evaluate_ins = EvaluateIns(parameters, config)

        # Return client configurations as a list of (ClientProxy, EvaluateIns)
        return [(client, evaluate_ins) for client in clients if client is not None]

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[BaseException],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""
        if not results:
            return None, {}

        # Total number of examples used for training
        total_examples = sum([res.num_examples for _, res in results])

        # Initialize the list to store weighted updates
        weighted_updates = []

        for _, fit_res in results:
            # Deserialize parameters to ndarrays
            client_weights = parameters_to_ndarrays(fit_res.parameters)
            # Calculate the weight based on the number of examples
            num_examples = fit_res.num_examples
            weight = num_examples / total_examples
            # Append the weighted client weights
            weighted_updates.append([layer * weight for layer in client_weights])

        # Sum the weighted updates
        aggregated_weights = [
            np.sum([update[layer] for update in weighted_updates], axis=0)
            for layer in range(len(weighted_updates[0]))
        ]

        # Serialize aggregated weights back to Parameters
        aggregated_parameters = ndarrays_to_parameters(aggregated_weights)

        # Return aggregated parameters and an empty metrics dictionary
        return aggregated_parameters, {}

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[BaseException],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation results."""
        if not results:
            return None, {}

        # Compute weighted loss
        losses = [res.num_examples * res.loss for _, res in results]
        examples = [res.num_examples for _, res in results]
        aggregated_loss = sum(losses) / sum(examples)

        # Compute weighted accuracy
        accuracies = [
            res.num_examples * res.metrics.get("accuracy", 0.0)
            for _, res in results
        ]
        aggregated_accuracy = sum(accuracies) / sum(examples)
        self.global_accuracy = aggregated_accuracy  # Store global accuracy

        # Update client accuracies
        for client_proxy, eval_res in results:
            cid = client_proxy.cid
            accuracy = eval_res.metrics.get("accuracy", 0.0)
            prev_accuracy = self.client_accuracies.get(cid, 0.0)
            self.client_accuracies[cid] = accuracy

            # Log client accuracies
            log(
                INFO,
                f"Client {cid}: accuracy = {accuracy:.4f} (prev: {prev_accuracy:.4f})",
            )

        # Log aggregated accuracy
        log(
            INFO,
            f"Round {server_round} aggregated accuracy: {aggregated_accuracy:.4f}\n",
        )

        # Print total communications after the last round
        if server_round == self.total_rounds:
            log(
                INFO,
                f"[SUMMARY]\nTotal communication rounds: {self.total_communications}",
            )

        # Return aggregated loss and metrics
        return aggregated_loss, {"accuracy": aggregated_accuracy}

    def evaluate(
        self,
        server_round: int,
        parameters: Parameters,
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate model parameters using an optional validation function."""
        if self.evaluate_fn is None:
            # No validation function provided
            return None
        return self.evaluate_fn(server_round, parameters)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def run_simulation(k: int, alpha: float, initial_rounds: int, total_rounds: int) -> Dict:
    """
    Runs a single federated learning simulation with given k and alpha.

    Args:
        k (int): Number of clients to select per round.
        alpha (float): Smoothing factor for selection probability.
        initial_rounds (int): Number of initial rounds to enforce top-k selection.
        total_rounds (int): Total number of communication rounds.

    Returns:
        Dict: A dictionary containing k, alpha, final_accuracy, and total_communications.
    """
    print(f"\nRunning simulation with k={k}, alpha={alpha}")

    # Initialize the strategy with current k and alpha
    strategy = CS_final(
        fraction_fit=1.0,
        fraction_evaluate=1.0,
        min_fit_clients=2,
        min_evaluate_clients=2,
        min_available_clients=2,
        initial_rounds=initial_rounds,
        k=k,
        alpha=alpha,
        total_rounds=total_rounds,
    )

    # Start the Flower simulation
    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=NUM_CLIENTS,
        config=fl.server.ServerConfig(num_rounds=total_rounds),
        strategy=strategy,
    )

    # After simulation, retrieve the metrics
    final_accuracy = strategy.global_accuracy
    total_communications = strategy.total_communications

    print(f"Completed simulation with k={k}, alpha={alpha}: Accuracy={final_accuracy}, Communications={total_communications}")

    return {
        'k': k,
        'alpha': alpha,
        'final_accuracy': final_accuracy,
        'total_communications': total_communications
    }

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Scatter Plot

In [ ]:

import plotly.express as px
import plotly.graph_objects as go

def main():

    k_values = [2, 3, 5, 7]
    alpha_values = [0.1, 0.2, 0.5, 0.8]

    # Initialize a list to store experiment results
    experiment_results = []

    # Iterate over all combinations of k and alpha
    for k, alpha in itertools.product(k_values, alpha_values):
        result = run_simulation(
            k=k,
            alpha=alpha,
            initial_rounds=INITIAL_ROUNDS,
            total_rounds=NUM_ROUNDS
        )
        experiment_results.append(result)

    # Save the results to a CSV file
    csv_file = 'experiment_results.csv'
    df = pd.DataFrame(experiment_results)
    df.to_csv(csv_file, index=False)
    print(f"\nAll experiments completed. Results saved to {csv_file}")

    # Proceed to visualization
    visualize_results(df)

def visualize_results(df: pd.DataFrame):


    # Create a label for legend entries
    df['params'] = df.apply(lambda row: f"k={row['k']}, alpha={row['alpha']}", axis=1)

    #scatter plot
    fig = px.scatter(
        df,
        x='final_accuracy',
        y='total_communications',
        color='params',
        hover_data={
            'k': True,
            'alpha': True,
            'final_accuracy': ':.4f',
            'total_communications': True
        },
        labels={
            'final_accuracy': 'Final Accuracy',
            'total_communications': 'Total Communication Rounds',
            'params': 'Parameters'
        },
        title='Federated Learning Performance: Accuracy vs Communication Rounds'
    )


    fig.update_traces(
        marker=dict(
            size=12,
            opacity=0.8,
            line=dict(width=1, color='DarkSlateGrey')
        )
    )


    fig.update_layout(
        legend_title_text='Parameters',
        xaxis_title='Final Accuracy',
        yaxis_title='Total Communication Rounds',
        hovermode='closest',
        margin=dict(l=60, r=200, t=60, b=60),
        legend=dict(
            x=1.05,
            y=1,
            xanchor='left',
            yanchor='top',
            bgcolor='rgba(255,255,255,0.9)',
            bordercolor='rgba(0,0,0,0.1)',
            borderwidth=1
        )
    )

    # Save to HTML and show
    fig.write_html('interactive_plot.html')
    fig.show()


if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=20, no round_timeout



Running simulation with k=2, alpha=0.1


2024-12-28 01:37:39,872	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'object_store_memory': 3985032806.0, 'memory': 7970065614.0, 'CPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=51862) 2024-12-28 01:37:46.908392: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=51862) 2024-12-28 01:37:46.953370: E external/local_xla/xla/s

Completed simulation with k=2, alpha=0.1: Accuracy=0.8242638837122624, Communications=40

Running simulation with k=2, alpha=0.2


2024-12-28 01:43:28,481	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7982857422.0, 'object_store_memory': 3991428710.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=53681) 2024-12-28 01:43:33.163172: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=53681) 2024-12-28 01:43:33.203365: E external/local_xla/xla/s

...

Completed simulation with k=7, alpha=0.2: Accuracy=0.9796869176295192, Communications=140

Running simulation with k=7, alpha=0.5


2024-12-28 03:34:16,094	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0, 'object_store_memory': 3983775744.0, 'memory': 7967551488.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=86556) 2024-12-28 03:34:20.492788: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=86556) 2024-12-28 03:34:20.534484: E external/local_xla/xla/s

Completed simulation with k=7, alpha=0.5: Accuracy=0.9651509504286246, Communications=140

Running simulation with k=7, alpha=0.8


2024-12-28 03:46:27,534	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3983674982.0, 'memory': 7967349966.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=90022) 2024-12-28 03:46:32.024245: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=90022) 2024-12-28 03:46:32.066501: E external/local_xla/xla/s

Completed simulation with k=7, alpha=0.8: Accuracy=0.9817368617219531, Communications=140

All experiments completed. Results saved to experiment_results.csv
